In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import os
import sys

notebook_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
print(sys.path)

%reload_ext autoreload
%autoreload 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

['/mnt/ceph/users/blyo1/projects', '/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python310.zip', '/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10', '/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/lib-dynload', '', '/mnt/home/blyo1/venvs/py310/lib/python3.10/site-packages']


# hVAE

In [14]:
import argparse
from b_models.vae import VariableConvEncoder, VariableConvDecoder, VAE, UNetDecoder

num_channels = 1
image_dims = 32
latent_dims = 5

enc = VariableConvEncoder(
    num_channels,
    image_dims,
    latent_dims, 
    channels=[32, 32, 32], 
    bias=True,
)

dec = VariableConvDecoder(
    num_channels,
    image_dims,
    latent_dims,
    channels=[32, 32, 32],  # output_channels, kernel_size, stride, padding
    output_channels=1,
    bias=True,
)

vae = VAE(
    encoder=enc,
    decoder=dec,
    kl_reduction="mean"
)

In [44]:
from b_models.vae import VariableConvEncoder

enc = VariableConvEncoder(num_channels=1, image_dims=32, latent_dims=5, channels=[4, 8, 16])
print(enc.d1_input_dims)
print(enc.d2_input_dims)
print(enc.d3_input_dims)

print(enc.conv_d1.weight.shape)
print(enc.conv_d2.weight.shape)
print(enc.conv_d3.weight.shape)

print(enc.linear_mu.weight.shape)

32
16
8
torch.Size([4, 1, 3, 3])
torch.Size([8, 4, 3, 3])
torch.Size([16, 8, 3, 3])
torch.Size([5, 256])


In [62]:
from b_models.hvae import EncoderConvBlock

example_input = torch.randn(1, 1, 32, 32)
enc_block = EncoderConvBlock(num_channels=1, img_dim=example_input.shape[3], latent_dim=5, channels=[4, 8, 16])

for i in [0, 2, 4]:
    print(enc_block.conv_network[i].weight.shape)
print(enc_block.linear_mu.weight.shape)
d, mu, var = enc_block(example_input)

torch.Size([4, 1, 3, 3])
torch.Size([8, 4, 3, 3])
torch.Size([16, 8, 3, 3])
torch.Size([5, 256])


In [94]:
from b_models.hvae import DecoderConvBlock

dec_block = DecoderConvBlock(output_dim=16, latent_dim=5, channels=[16, 8, 4])

example_input = torch.randn(1, 5)
dec_out = dec_block(example_input)
print(dec_out.shape)

# for i in range(len(dec_block.deconv_network)):
#     if hasattr(dec_block.deconv_network[i], 'weight'):
#         print(dec_block.deconv_network[i].weight.shape)

torch.Size([1, 4, 16, 16])


In [108]:
from b_models.hvae import DecoderConvBlock

dec_block = DecoderConvBlock(latent_in_dim=5, latent_out_dim=2, output_dim=8, channels=[16, 8, 4])

example_input = torch.randn(1, 5)
dec_out, _, _ = dec_block(example_input)
print(dec_out.shape)

for i in range(len(dec_block.deconv_network)):
    if hasattr(dec_block.deconv_network[i], 'weight'):
        print(dec_block.deconv_network[i].weight.shape)

torch.Size([1, 2])
torch.Size([4, 8, 4, 4])
torch.Size([8, 16, 4, 4])
torch.Size([16, 4, 4, 4])
